In [18]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from scipy.sparse import csr_matrix
import string
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import sklearn as sk
from sklearn.feature_extraction.text import TfidfVectorizer
import math
import time

def problemOne(file_name):
    doc = []
    file = open(file_name, 'r')
    for line in file:
        doc.append(line)
    
    cv = CountVectorizer(token_pattern=r"(?u)\b\w+\b")   
    count_occurrences = cv.fit_transform(doc)
    
    count_occurrences.toarray()
    
    count_vect_df = pd.DataFrame(data = count_occurrences.toarray(), columns=cv.get_feature_names_out())
    
    print(count_vect_df)
    
    
start = time.time()
problemOne('train.dat')
problemOne('test.dat')
print(time.time() - start)

102080
        0  00  000  000016447  00010563n  0002  0004287  0005173miami050  \
0       0   0    0          0          0     0        0                0   
1       0   0    0          0          0     0        0                0   
2       0   0    0          0          0     0        0                0   
3       0   0    0          0          0     0        0                0   
4       0   0    0          0          0     0        0                0   
...    ..  ..  ...        ...        ...   ...      ...              ...   
102075  0   0    0          0          0     0        0                0   
102076  0   0    0          0          0     0        0                0   
102077  0   0    0          0          0     0        0                0   
102078  0   0    0          0          0     0        0                0   
102079  0   0    0          0          0     0        0                0   

        0009875buffalo040  000bn  ...  zwick  zwiki  zy  zydrunas  zygmunt  \
0 

In [26]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from scipy.sparse import csr_matrix
import string
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import sklearn as sk
from sklearn.feature_extraction.text import TfidfVectorizer
import math
import time

def build_both_ids(both):
    r""" Build a mapping from word to ID and vice versa. 
    """
    both_idx = {}
    tid = 0
    for d in both:
        for w in d:
            w.lower()
            if w not in both_idx:
                both_idx[w] = tid
                tid += 1
    return both_idx

def build_matrix(docs, idx):
    r""" Build sparse matrix from a list of documents, 
    each of which is a list of word/terms in the document.  
    """
    nrows = len(docs)
    # tid = 0
    nnz = 0
    for d in docs:
        nnz += len(set(d))
    #     for w in d:
    #         w.lower()
    #         if w not in idx:
    #             idx[w] = tid
    #             tid += 1
    ncols = len(idx)
        
    # set up memory
    ind = np.zeros(nnz, dtype=int)
    val = np.zeros(nnz, dtype=np.double)
    ptr = np.zeros(nrows+1, dtype=int)
    i = 0  # document ID / row counter
    n = 0  # non-zero counter
    # transfer values
    for d in docs:
        cnt = Counter(d)
        keys = list(k for k,_ in cnt.most_common())
        l = len(keys)
        for j,k in enumerate(keys):
            ind[j+n] = idx[k]
            val[j+n] = cnt[k]
        ptr[i+1] = ptr[i] + l
        n += l
        i += 1
            
    mat = csr_matrix((val, ind, ptr), shape=(nrows, ncols), dtype=np.double)
    mat.sort_indices()
    
    return mat

def csr_info(mat, name="", non_empy=False):
    r""" Print out info about this CSR matrix. If non_empy, 
    report number of non-empty rows and cols as well
    """
    if non_empy:
        print("%s [nrows %d (%d non-empty), ncols %d (%d non-empty), nnz %d]" % (
                name, mat.shape[0], 
                sum(1 if mat.indptr[i+1] > mat.indptr[i] else 0 
                for i in range(mat.shape[0])), 
                mat.shape[1], len(np.unique(mat.indices)), 
                len(mat.data)))
    else:
        print( "%s [nrows %d, ncols %d, nnz %d]" % (name, 
                mat.shape[0], mat.shape[1], len(mat.data)) )

def csr_l2normalize(mat, copy=False, **kargs):
    r""" Normalize the rows of a CSR matrix by their L-2 norm. 
    If copy is True, returns a copy of the normalized matrix.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # normalize
    for i in range(nrows):
        rsum = 0.0    
        for j in range(ptr[i], ptr[i+1]):
            rsum += val[j]**2
        if rsum == 0.0:
            continue  # do not normalize empty rows
        rsum = 1.0/np.sqrt(rsum)
        for j in range(ptr[i], ptr[i+1]):
            val[j] *= rsum
            
    if copy is True:
        return mat
        
# def textToMatrix(file, idx):
#     docs = [l.split() for l in file]
#     return build_matrix(docs, idx)

def both(file1, file2):
    docs1 = [l.split() for l in file1]
    docs2 = [l.split() for l in file2]
    doc_both = docs1 + docs2
    return docs1, docs2, doc_both

start = time.time()
test = open('test.dat', 'r')
train = open('train.dat', 'r')

docs1, docs2, doc_both = both(test, train)
idx = build_both_ids(doc_both)
csr_info(build_matrix(docs1, idx))
csr_info(build_matrix(docs2, idx))

print(time.time() - start)

 [nrows 25520, ncols 194175, nnz 849425]
 [nrows 102080, ncols 194175, nnz 3497809]
5.028883934020996
